In [1]:
import os
import pandas as pd
import re
import numpy as np
from datetime import datetime
from openpyxl import load_workbook
import warnings


warnings.filterwarnings("ignore", category=UserWarning, module='openpyxl')


# input

your_ing_id = 'XO21BM'
date = datetime.strptime('2024', "%Y")

directory_path0 = os.getcwd()
directory_path = path = f"/Users/{your_ing_id}/Library/CloudStorage/OneDrive-SharedLibraries-ING/Product Evaluation and Risk Assessment Library (PEARL) - PEARL_Repository"
directory_path2 = path = f"/Users/{your_ing_id}/Library/CloudStorage/OneDrive-SharedLibraries-ING/Product Evaluation and Risk Assessment Library (PEARL) - MI Dashboard"        


In [2]:
directory_path2

'/Users/XO21BM/Library/CloudStorage/OneDrive-SharedLibraries-ING/Product Evaluation and Risk Assessment Library (PEARL) - MI Dashboard'

In [3]:
pwd

'/Users/xo21bm/Documents/NFR'

In [166]:
# folder and ID mapping
overview = pd.read_csv('PEARL List (1).csv', sep=',', low_memory=False)
overview['Folder_URL_txt'] = overview['Folder_URL_txt'].str.replace('#$@', ' ')
overview['Folder'] = overview['Folder_URL_txt'].str.replace(r'https://ing.sharepoint.com/sites/PEARL_cs/Shared Documents/Pearl_Repository/', '')
overview['Folder'] = overview['Folder'].str.replace(r'https://ing.sharepoint.com/sites/PEARL_cs/Shared Documents/PEARL_Repository/', '')
cols_overview = ['Folder', 'ID']
overview = overview[cols_overview]
overview['ID'] = overview['ID'].astype(str)



In [144]:
overview

,Folder,ID
0,Sparen/Oranje Spaarrekening/2021-10-29 NIR com...,184
1,Current accounts/Betaalrekening/2021-11-02 LIT...,185
2,Hypotheken - WUB/Z_Other/2021-11-04 Vervallen ...,188
3,NaN,189
4,Current accounts/Zakelijke rekening/2021-11-03...,190
...,...,...
2997,Payments_Giraal betvk/SEPA Direct Debit (credi...,3883
2998,Beleggingsrekeningen/Eenvoudig Beleggen/2025-0...,3884
2999,Verzekeren/New Product/2025-04-16 ING Income P...,3885
3000,KYC/Process change/2025-04-16 STP 2.0,3886


In [167]:
# reading MI dashboard
file_name = 'MI Dashboard.xlsm'
file_path = os.path.join(directory_path2, file_name)

In [168]:
# Changes
mi_dash_changes = pd.read_excel(file_path, sheet_name="Management Info Changes", engine="openpyxl", skiprows=2)

# filtering only the relevant ones for our current analysis on the complete ones
mi_dash_changes.loc[mi_dash_changes['End Date'] == '(blank)', 'End Date'] = np.nan
mi_dash_changes = mi_dash_changes[mi_dash_changes['End Date'] >= date]
mi_dash_changes = mi_dash_changes[mi_dash_changes['Status'] == '5. Journey finished']
#mi_dash_changes = mi_dash_changes[mi_dash_changes['Process Category'] != 'Communication'] # according to Hasan

cols_changes = ['PEARL ID', 'Start Date', 'End Date', 'Process Category', 'Tribe']
mi_dash_changes = mi_dash_changes[cols_changes]
mi_dash_changes.rename(columns={'PEARL ID': 'ID'}, inplace=True)
mi_dash_changes = mi_dash_changes.merge(overview, how = 'left', on = 'ID')
mi_dash_changes['Type'] = 'Change'

In [170]:
# Reviews
mi_dash_review = pd.read_excel(file_path, sheet_name="Management Info Reviews", engine="openpyxl", skiprows=2)
mi_dash_review.rename(columns={'RJT Review PEARL ID': 'ID', 
                               'RJT Review Start Date': 'Start Date', 
                               'RJT Review Approval Date': 'End Date'}, inplace=True)

# filtering only the relevant ones for our current analysis on the complete ones
mi_dash_review.loc[mi_dash_review['End Date'] == '(blank)', 'End Date'] = np.nan
mi_dash_review = mi_dash_review[mi_dash_review['End Date'] >= date]
mi_dash_review = mi_dash_review[mi_dash_review['RJT Review Status'] == '5. Journey finished']

cols_review = ['ID', 'Start Date', 'End Date']
mi_dash_review = mi_dash_review[cols_review]
mi_dash_review = mi_dash_review.merge(overview, how = 'left', on = 'ID')
mi_dash_review['Type'] = 'Review'

# creating overview so we know in which risk assesments we are interested in
final_overview = pd.concat([mi_dash_changes, mi_dash_review])

In [171]:
final_overview 

,ID,Start Date,End Date,Process Category,Tribe,Folder,Type
0,1008,2022-09-15 00:00:00,2024-07-11 00:00:00,Communication,Tribe Digital & Customer Interactions,Digital Sales/Consumer Loans/2022-09-15 Care j...,Change
1,1037,2022-09-23 00:00:00,2024-10-30 00:00:00,Significant change financial product/-service/...,Consumer Lending & Insurance,Doorlopend Krediet/Studentenkrediet/2022-09-23...,Change
2,1050,2022-09-29 00:00:00,2024-04-08 00:00:00,Significant change financial product/-service/...,Tribe Payments,Payments_Giraal betvk/iDEAL 2.0/2022-09-29 iDE...,Change
3,1060,2022-09-30 00:00:00,2024-03-28 00:00:00,Other change (non-PAP),Tribe Investments,Beleggingsrekeningen/Vermogensbeheer/2022-09-3...,Change
4,1068,2022-10-04 00:00:00,2024-02-07 00:00:00,Other change (non-PAP),Tribe Payments,Payments Cards Issuing/Betaalpas zakelijk (car...,Change
...,...,...,...,...,...,...,...
45,3307,2024-10-28 00:00:00,2025-02-26 00:00:00,NaN,NaN,Doorlopend Krediet/Review multiple products/20...,Review
46,3307,2024-10-28 00:00:00,2025-02-26 00:00:00,NaN,NaN,Doorlopend Krediet/Review multiple products/20...,Review
47,3230,2024-10-08 00:00:00,2025-03-25 00:00:00,NaN,NaN,Payments Reporting/Afschriften (papier)/2024-1...,Review
48,3455,2024-12-05 00:00:00,2025-01-29 00:00:00,NaN,NaN,Vendor Leases/Review multiple products/2024-12...,Review


In [172]:
# prep for the loop
result1 = pd.DataFrame()
result2 = pd.DataFrame()
result3 = pd.DataFrame()

In [173]:
def find_matching_strings(strings):
    matching_strings = []
    for string in strings:
        if string.lower().startswith('risk') and 'journey' in string.lower() and string.lower().endswith('xlsm'):
            matching_strings.append(string)
    
    return matching_strings

folders_to_check = final_overview.loc[:, 'Folder']



In [174]:
folders_to_check

0     Digital Sales/Consumer Loans/2022-09-15 Care j...
1     Doorlopend Krediet/Studentenkrediet/2022-09-23...
2     Payments_Giraal betvk/iDEAL 2.0/2022-09-29 iDE...
3     Beleggingsrekeningen/Vermogensbeheer/2022-09-3...
4     Payments Cards Issuing/Betaalpas zakelijk (car...
                            ...                        
45    Doorlopend Krediet/Review multiple products/20...
46    Doorlopend Krediet/Review multiple products/20...
47    Payments Reporting/Afschriften (papier)/2024-1...
48    Vendor Leases/Review multiple products/2024-12...
49    Vendor Leases/Review multiple products/2024-12...
Name: Folder, Length: 870, dtype: object

In [175]:
# for testing and cheking specific folder ########
item_to_find = r'Hypotheken - WUB/WUB Hypotheek met Beleggingsrekening/2024-04-15 Intermediate Review product'
temp_index = 0
for f in folders_to_check:
    
    if f == item_to_find:
        break
    
    temp_index = temp_index + 1
    
folder = folders_to_check.iloc[temp_index] # for testing
folder = folders_to_check.iloc[859] # for testing

In [159]:
folders_to_check

0     Digital Sales/Consumer Loans/2022-09-15 Care j...
1     Doorlopend Krediet/Studentenkrediet/2022-09-23...
2     Payments_Giraal betvk/iDEAL 2.0/2022-09-29 iDE...
3     Beleggingsrekeningen/Vermogensbeheer/2022-09-3...
4     Payments Cards Issuing/Betaalpas zakelijk (car...
                            ...                        
45    Doorlopend Krediet/Review multiple products/20...
46    Doorlopend Krediet/Review multiple products/20...
47    Payments Reporting/Afschriften (papier)/2024-1...
48    Vendor Leases/Review multiple products/2024-12...
49    Vendor Leases/Review multiple products/2024-12...
Name: Folder, Length: 860, dtype: object

In [179]:
from pathlib import Path
import pandas as pd

index = 0  # Counter to track loop progress

for folder in folders_to_check:
    folder = str(folder)  # Convert to string in case it's a float
    folder = folder.replace("\\", "/")  # Normalize slashes for macOS/Linux
    complete_folder_path = Path(directory_path) / folder  # Use pathlib for safe path handling
    index += 1  # Increment counter

    print(f'Processing Folder {index}: {folder}')
    
    if complete_folder_path.is_dir():
        files = [f.name for f in complete_folder_path.iterdir() if f.is_file()]  # List files safely
        files = find_matching_strings(files)  # Apply matching function
    else:
        continue  # Skip non-existent directories
    
    for f in files:  # Loop through matching Excel files
        file_path = complete_folder_path / f  # Correctly construct file path
        try:
            xls = pd.ExcelFile(file_path, engine="openpyxl")
            sheet_names = xls.sheet_names
        except Exception:
            print(f'Error: Could not open {file_path}')
            continue
        
        # Process "Process & Module Selection"
        if "Process & Module Selection" in sheet_names:
            try:
                df = pd.read_excel(file_path, sheet_name="Process & Module Selection", engine="openpyxl")
            except Exception:
                print(f'Error reading {file_path}')
                continue
            
            row_index = df.index[df.iloc[:,1] == 'Risk entity']
            if row_index.empty:
                print(f'Missing Risk entity row in {file_path}')
                continue
            
            orm_row_index = row_index.tolist()[0] + 1
            irm_row_index = row_index.tolist()[0] + 4
            data = pd.DataFrame({
                'File': [f.lower()], 'ORM_check': [df.iloc[orm_row_index, 1]],
                'ORM': [df.iloc[orm_row_index, 5]], 'IRM_check': [df.iloc[irm_row_index, 1]],
                'IRM': [df.iloc[irm_row_index, 5]], 'Folder': [folder]
            })
            result1 = pd.concat([data, result1])

        # Process "General Risk Identification"
        if "General Risk Identification" in sheet_names:
            try:
                df = pd.read_excel(file_path, sheet_name="General Risk Identification", engine="openpyxl")
            except Exception:
                print(f'Error reading {file_path}')
                continue
            
            row_index = df.index[df.iloc[:,6] == 'Must be invited?']
            if row_index.empty:
                print(f'Missing "Must be invited" row in {file_path}')
                continue
            
            orm_row_index = row_index.tolist()[0] + 1
            irm_row_index = row_index.tolist()[0] + 4
            data = pd.DataFrame({
                'File': [f.lower()], 'ORM_check': [df.iloc[orm_row_index, 1]],
                'ORM_invited': [df.iloc[orm_row_index, 6]], 
                'ORM_part_of_risk_asses': [df.iloc[orm_row_index, 9]],
                'IRM_check': [df.iloc[irm_row_index, 1]], 
                'IRM_invited': [df.iloc[irm_row_index, 6]], 
                'IRM_part_of_risk_asses': [df.iloc[irm_row_index, 9]], 
                'Folder': [folder]
            })
            result2 = pd.concat([data, result2])

        # Process "Risk Summary & Approval"
        if "Risk Summary & Approval" in sheet_names:
            try:
                df = pd.read_excel(file_path, sheet_name="Risk Summary & Approval", engine="openpyxl")
            except Exception:
                print(f'Error reading {file_path}')
                continue
            
            row_index = df.index[df.iloc[:,1] == 'Step 4 -  Select Risk Entities to inform or to invite for challenge and agreement']
            if row_index.empty:
                continue  # Skip missing step (too common)
            
            orm_row_index = row_index.tolist()[0] + 3
            irm_row_index = row_index.tolist()[0] + 6
            data = pd.DataFrame({
                'File': [f.lower()], 'ORM_check': [df.iloc[orm_row_index, 1]],
                'ORM_invited_or_challange': [df.iloc[orm_row_index, 4]], 
                'IRM_check': [df.iloc[irm_row_index, 1]], 
                'IRM_invited_or_challange': [df.iloc[irm_row_index, 4]], 
                'Folder': [folder]
            })
            result3 = pd.concat([data, result3])

# Normalize folder paths before merging
for result in [result1, result2, result3]:
    result['Folder'] = result['Folder'].str.replace("\\", "/")

# Merge with final overview
result1 = result1.merge(final_overview, how='right', on='Folder')
result2 = result2.merge(final_overview, how='right', on='Folder')
result3 = result3.merge(final_overview, how='right', on='Folder')

# Save results
result1.to_csv('result1_intermediate.csv', sep=';', index=False)
result2.to_csv('result2_intermediate.csv', sep=';', index=False)
result3.to_csv('result3_intermediate.csv', sep=';', index=False)


Processing Folder 1: Digital Sales/Consumer Loans/2022-09-15 Care je lening staat klaar (verbouwen)
Processing Folder 2: Doorlopend Krediet/Studentenkrediet/2022-09-23 Simplify Loans products (Cancel Student Loan)
Processing Folder 3: Payments_Giraal betvk/iDEAL 2.0/2022-09-29 iDEAL 2.0 Release 3 Acquiring
Processing Folder 4: Beleggingsrekeningen/Vermogensbeheer/2022-09-30 Aanpassing revisie beleid en online revisie flow Vermogensbeheer
Processing Folder 5: Payments Cards Issuing/Betaalpas zakelijk (cards issuing)/2022-10-04 Soft block business (AppMINGZ)
Processing Folder 6: Private Banking _ Wealth/Campagnes/2022-10-05 Award campaign ING Private Banking 2022
Processing Folder 7: Persoonlijke Lening/Persoonlijke Lening/2022-10-11 Transactional scorecard NL for internal ING customers  EPIC0081965
Processing Folder 8: Digital Sales/Z_Other/2024 -10-28 Linkbuilding
Processing Folder 9: Beleggingsrekeningen/Beleggen App/2022-11-22 Small CX improvements in the Mobile Investments App
Proce